## Plan and solve 优化与测试

### 输出渲染测试

In [20]:
from Plan_and_solve import PlanAndSolveAgent

agent = PlanAndSolveAgent(llm_client)

question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
agent.run(question)

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


In [ ]:
from Plan_and_solve import PlanAndSolveAgent

agent = PlanAndSolveAgent(llm_client)

# 一个注定会引发争议/错误的简单数学题
question = "计算 10 除以 0 的结果，然后将结果加上 5。"
agent.run(question)

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


### 动态重规划 (Replanning) 机制引入

In [21]:
import sys
import os
import ast
from typing import List, Tuple

from llm_client import HelloAgentsLLM
from ui_utils import print_html

# --- 1. 增强版规划器 (Planner) ---
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划，```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""

REPLAN_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。智能体在执行某个计划时遇到了阻碍，需要你协助进行动态重规划。

# 原始问题:
{question}

# 已完成的历史步骤与结果:
{history}

# 遇到的问题/阻碍:
{failure_reason}

你的任务是：基于目前的状况，为**剩余未完成的目标**生成一个新的行动计划。
请不要包含已经完成的步骤，只输出接下来的步骤。

请严格按照以下格式输出你的新计划，```python与```作为前后缀是必要的:
```python
["新步骤1", "新步骤2", ...]
```
"""

class ReplanningPlanner:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def _parse_plan(self, response_text: str) -> list[str]:
        try:
            if "```python" in response_text:
                plan_str = response_text.split("```python")[1].split("```")[0].strip()
            elif "```" in response_text:
                 plan_str = response_text.split("```")[1].split("```")[0].strip()
            else:
                plan_str = response_text.strip()
            
            plan = ast.literal_eval(plan_str)
            return plan if isinstance(plan, list) else []
        except Exception as e:
            print_html(f"解析计划出错: {e}", title="❌ Parse Error")
            return []

    def plan(self, question: str) -> list[str]:
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        messages = [{"role": "user", "content": prompt}]
        
        print_html("正在生成初始计划...", title="🧠 Planner Thinking")
        response_text = self.llm_client.think(messages=messages) or ""
        
        plan = self._parse_plan(response_text)
        if plan:
            print_html(plan, title="✅ Initial Plan Generated")
        return plan

    def replan(self, question: str, history: str, failure_reason: str) -> list[str]:
        prompt = REPLAN_PROMPT_TEMPLATE.format(
            question=question,
            history=history,
            failure_reason=failure_reason
        )
        messages = [{"role": "user", "content": prompt}]
        
        print_html("正在进行动态重规划...", title="🔄 Replanning Thinking")
        response_text = self.llm_client.think(messages=messages) or ""
        
        new_plan = self._parse_plan(response_text)
        if new_plan:
            print_html(new_plan, title="✅ New Plan Generated")
        return new_plan

# --- 2. 增强版执行器 (Executor) ---
EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请仅输出针对“当前步骤”的回答:
"""

class ReplanningExecutor:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def execute_step(self, question: str, plan: list[str], history: str, current_step: str) -> str:
        prompt = EXECUTOR_PROMPT_TEMPLATE.format(
            question=question, 
            plan=plan, 
            history=history if history else "无", 
            current_step=current_step
        )
        messages = [{"role": "user", "content": prompt}]
        return self.llm_client.think(messages=messages) or ""

    def evaluate_result(self, step: str, result: str) -> Tuple[bool, str]:
        """
        简单的评估函数，用于判断执行结果是否成功。
        在真实场景中，这可能是一个独立的 LLM 调用或规则检查。
        """
        # 模拟：如果结果中包含 "无法"、"不知道"、"Error" 等词，视为失败
        failure_keywords = ["无法", "不知道", "error", "unknown", "failed"]
        if any(k in result.lower() for k in failure_keywords):
            return False, f"执行结果包含错误关键词，可能未正确完成: {result}"
        
        # 模拟：为了演示重规划，我们可以强制让特定步骤失败
        # 例如：如果步骤包含 "周二"，我们假装它失败了（用于演示）
        if "周二" in step and "模拟失败" not in result: # 防止死循环
             # 这里只是为了演示效果，实际使用请删除此逻辑
             # return False, "模拟：获取周二数据时连接超时 (Simulated Failure)"
             pass

        return True, "Success"

# --- 3. 支持重规划的智能体 ---
class ReplanningAgent:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client
        self.planner = ReplanningPlanner(self.llm_client)
        self.executor = ReplanningExecutor(self.llm_client)

    def run(self, question: str):
        print_html(question, title="🏁 Task Start (With Replanning)")
        
        # 1. 初始规划
        plan = self.planner.plan(question)
        if not plan:
            print_html("无法生成初始计划。", title="🛑 Stop")
            return

        history_str = ""
        final_answer = ""
        
        # 使用 while 循环来支持动态修改 plan 列表
        while plan:
            current_step = plan.pop(0) # 取出当前第一步
            
            # 执行
            result = self.executor.execute_step(question, plan, history_str, current_step)
            
            # 渲染执行结果
            print_html(f"**Step**: {current_step}\n**Result**: {result}", title="🎬 Executing Step")

            # 评估
            success, message = self.executor.evaluate_result(current_step, result)
            
            if success:
                # 成功：记录历史，继续
                history_str += f"步骤: {current_step}\n结果: {result}\n\n"
                final_answer = result
            else:
                # 失败：触发重规划
                print_html(f"检测到执行异常: {message}", title="⚠️ Execution Failed")
                
                # 调用重规划，获取新的后续步骤
                new_remaining_plan = self.planner.replan(question, history_str, failure_reason=message)
                
                if new_remaining_plan:
                    # 用新计划替换剩余的旧计划
                    plan = new_remaining_plan 
                    print_html(f"已更新后续计划: {plan}", title="🔄 Plan Updated")
                else:
                    print_html("重规划失败，任务终止。", title="🛑 Stop")
                    return

        print_html(final_answer, title="🎉 Final Answer")


#### 测试（一）

In [ ]:
# --- 运行测试 ---相同问题
if __name__ == "__main__":
    try:
        from dotenv import load_dotenv
        load_dotenv()
        
        llm = HelloAgentsLLM()
        agent = ReplanningAgent(llm)
        
        # 我们可以尝试一个稍微复杂点的问题，或者沿用之前的问题
        # 为了触发可能的错误，可以尝试问一些模型可能不知道的实时信息，或者逻辑陷阱
        question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
        
        agent.run(question)
        
    except Exception as e:
        print_html(f"Error: {e}", title="❌ System Error")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


#### 测试（二）

In [ ]:
# --- 运行测试 ---动态重规划机制测试问题
if __name__ == "__main__":
    try:
        from dotenv import load_dotenv
        load_dotenv()
        llm = HelloAgentsLLM()
        
        agent = ReplanningAgent(llm)
        
        # 一个注定会引发争议/错误的简单数学题
        question = "计算 10 除以 0 的结果，然后将结果加上 5。"
        
        agent.run(question)
        
    except Exception as e:
        print_html(f"Error: {e}", title="❌ System Error")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


...

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


这里的多次重规划现象非常有趣，它生动地展示了**基于规则（关键词）的评估机制的局限性**。

————现象还原

在计算 `10/0` 的任务中，Agent 陷入了 **"执行 -> 判错 -> 重规划 -> 再执行 -> 再判错"** 的循环：

1.  **第一轮（正确触发）**：
    *   **动作**：执行计算 `10/0`。
    *   **结果**：LLM 返回 "无法计算..."。
    *   **评估**：检测到关键词 `无法`，判定为 **失败**。
    *   **重规划**：Agent 很聪明，生成新计划——“报告数学错误”。

2.  **第二轮（误判）**：
    *   **动作**：执行新计划“报告数学错误”。
    *   **结果**：LLM 解释道 "因为除数为0，所以**无法**计算..."。
    *   **评估**：**关键点在这里！** 评估器再次检测到结果中包含 `无法` 这个词。
    *   **判定**：评估器“死板”地认为任务又失败了，再次触发重规划。

————核心原因分析

我们在 `ReplanningExecutor` 中实现的是一个**简单的关键词匹配评估器**：

```python
# /agent/实战调试.ipynb 的 evaluate_result 函数
failure_keywords = ["无法", "不知道", "error", "unknown", "failed"]
if any(k in result.lower() for k in failure_keywords):
    return False, f"执行结果...: {result}"
```

这个评估器**不懂语义**，它分不清：
*   ❌ **执行失败**："我尝试计算了，但系统报错，无法得出结果。"
*   ✅ **合理的负面结论**："经过分析，这个数学题本身是错误的，因此无法计算。"

————改进启示

这个“事故”为我们设计更强大的 Agent 提供了重要经验：
*   **规则评估的局限**：简单的关键词匹配容易导致“误杀”。
*   **需要语义评估 (Critic)**：在生产环境中，`evaluate_result` 不应只是正则匹配，而应该是一个 **LLM 调用**。让另一个 LLM（裁判）来判断："这个结果虽然包含'无法'，但它是否圆满回答了用户的疑问？"

如果把评估函数改成 LLM 调用，它就能读懂 "无法计算是由于除数为0"，从而判定任务 **成功**，终止不必要的重规划循环。

#### 语义评估 (Critic)引入

In [28]:
import json
import ast
from typing import List, Tuple

from llm_client import HelloAgentsLLM
from ui_utils import print_html
from dotenv import load_dotenv

# --- Prompts ---
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划，```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""

REPLAN_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。智能体在执行某个计划时遇到了阻碍，需要你协助进行动态重规划。

# 原始问题:
{question}

# 已完成的历史步骤与结果:
{history}

# 遇到的问题/阻碍:
{failure_reason}

你的任务是：基于目前的状况，为**剩余未完成的目标**生成一个新的行动计划。
请不要包含已经完成的步骤，只输出接下来的步骤。

请严格按照以下格式输出你的新计划，```python与```作为前后缀是必要的:
```python
["新步骤1", "新步骤2", ...]
```
"""

EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请仅输出针对“当前步骤”的回答:
"""

EVAL_PROMPT_TEMPLATE = """
你是一个客观的裁判。请判断以下任务步骤的执行结果是否成功。

任务步骤: {step}
执行结果: {result}

判断标准：
1. 如果结果是一个明确的答案（即使答案是"无解"、"不存在"，只要是基于事实的合理结论），视为【成功】。
2. 如果结果表明发生了系统错误、工具调用失败、或者回答"我不知道"且没有给出理由，视为【失败】。
3. 特别注意：如果结果解释了为什么无法计算（例如"除数不能为零"），这属于成功的回答。

请只输出 JSON 格式，格式如下：
```json
{{
    "success": true/false,
    "reason": "简短的理由"
}}
```
"""

# --- Classes ---

class ReplanningPlanner:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def _parse_plan(self, response_text: str) -> list[str]:
        try:
            if "```python" in response_text:
                plan_str = response_text.split("```python")[1].split("```")[0].strip()
            elif "```" in response_text:
                plan_str = response_text.split("```")[1].split("```")[0].strip()
            else:
                plan_str = response_text.strip()
            
            plan = ast.literal_eval(plan_str)
            if isinstance(plan, list):
                return plan
            return []
        except Exception:
            return []

    def plan(self, question: str) -> list[str]:
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        messages = [{"role": "user", "content": prompt}]
        
        print_html("正在生成初始计划...", title="🧠 Planner Thinking")
        response = self.llm_client.think(messages=messages) or ""
        
        plan = self._parse_plan(response)
        if plan:
            print_html(plan, title="✅ Plan Generated")
        else:
            print_html(f"无法生成计划。响应: {response}", title="❌ Plan Error")
            
        return plan

    def replan(self, question: str, history: str, failure_reason: str) -> list[str]:
        prompt = REPLAN_PROMPT_TEMPLATE.format(
            question=question,
            history=history,
            failure_reason=failure_reason
        )
        messages = [{"role": "user", "content": prompt}]
        
        print_html("正在进行动态重规划...", title="🔄 Replanning Thinking")
        response = self.llm_client.think(messages=messages) or ""
        
        plan = self._parse_plan(response)
        return plan

class ReplanningExecutor:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def execute_step(self, question: str, plan: list[str], history: str, current_step: str) -> str:
        prompt = EXECUTOR_PROMPT_TEMPLATE.format(
            question=question, 
            plan=plan, 
            history=history if history else "无", 
            current_step=current_step
        )
        messages = [{"role": "user", "content": prompt}]
        return self.llm_client.think(messages=messages) or ""

    def evaluate_result(self, step: str, result: str) -> Tuple[bool, str]:
        """
        使用 LLM (Critic) 进行语义评估
        """
        prompt = EVAL_PROMPT_TEMPLATE.format(step=step, result=result)
        messages = [{"role": "user", "content": prompt}]
        
        # print_html("正在评估执行结果...", title="⚖️ Critic Evaluating") 
        response = self.llm_client.think(messages=messages) or "{}"
        
        try:
            json_str = response
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
                
            eval_result = json.loads(json_str)
            return eval_result.get("success", False), eval_result.get("reason", "Unknown reason")
        except Exception as e:
            print(f"Critic parse error: {e}")
            return True, "Critic failed to parse, assuming success."

class ReplanningAgent:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client
        self.planner = ReplanningPlanner(self.llm_client)
        self.executor = ReplanningExecutor(self.llm_client)

    def run(self, question: str):
        print_html(question, title="🏁 Task Start (Pro Agent)")
        
        plan = self.planner.plan(question)
        if not plan:
            return

        history_str = ""
        final_answer = ""
        
        while plan:
            current_step = plan.pop(0)
            
            result = self.executor.execute_step(question, plan, history_str, current_step)
            print_html(f"**Step**: {current_step}\n**Result**: {result}", title="🎬 Executing Step")

            success, message = self.executor.evaluate_result(current_step, result)
            
            if success:
                history_str += f"步骤: {current_step}\n结果: {result}\n\n"
                final_answer = result
            else:
                print_html(f"Critic 判定失败: {message}", title="⚠️ Execution Rejected")
                
                new_remaining_plan = self.planner.replan(question, history_str, failure_reason=message)
                
                if new_remaining_plan:
                    plan = new_remaining_plan 
                    print_html(f"已更新后续计划: {plan}", title="🔄 Plan Updated")
                else:
                    print_html("重规划失败，任务终止。", title="🛑 Stop")
                    return

        print_html(final_answer, title="🎉 Final Answer")


#### 测试（三）

In [ ]:
# --- 运行测试 ---动态重规划机制测试问题
if __name__ == "__main__":
    load_dotenv()
    
    llm = HelloAgentsLLM()
    agent = ReplanningAgent(llm)
    
    # Test case: 10/0
    agent.run("计算 10 除以 0 的结果，然后将结果加上 5。")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


好的，非常高兴看到 **Critic（裁判）机制** 成功解决了死循环问题！这标志着我们的 Plan-and-Solve Agent 已经从一个“机械执行者”进化为了一个具有“自我反思与判断能力”的高级智能体。

以下是今天的会话总结报告：

---

**🏆 今日成果总结**

我们围绕 **Plan-and-Solve 范式** 进行了从基础实现到高级优化的完整迭代：

**1. 基础搭建：Plan-and-Solve 范式**
*   **实现**：创建了 `Plan_and_solve.py`，实现了经典的“先规划，后执行”架构。
*   **特点**：任务结构化清晰，适合解决步骤明确的复杂问题（如数学应用题）。
*   **局限**：静态规划一旦生成，执行过程中无法变通。

**2. 进阶优化：动态重规划 (Replanning)**
*   **痛点**：在 `实战调试.ipynb` 中，我们发现一旦中间步骤出错（如除以0），静态 Agent 会“硬着头皮”继续计算，导致结果荒谬。
*   **方案**：引入 **Monitor-Feedback Loop**。执行每一步后进行评估，若失败则触发 `replan`，让 Planner 重新生成剩余计划。
*   **效果**：Agent 学会了“知错能改”，遇到错误会尝试调整策略（例如从“计算”转为“报告错误”）。

**3. 终极完善：语义评估 (Critic)**
*   **痛点**：使用简单的关键词（如 `无法`、`Error`）进行评估会导致误判。Agent 正确地报告了“无法计算”，却被评估器误认为是执行失败，导致陷入无限重规划的死循环。
*   **方案**：实现 `Plan_and_solve_pro.py`，引入 **LLM Critic**。
    *   **核心逻辑**：不再查字典，而是让 LLM 理解语义。
    *   **Prompt 设计**：明确告诉 Critic，“基于事实的否定回答（如无解）”也是一种**成功**。
*   **结果**：完美解决了死循环问题。Agent 既能灵活应对错误，又能准确判断任务是否已合理终结。

---

**🧠 关键技术洞察**

1.  **静态 vs 动态**：
    *   静态 Plan-and-Solve 适合环境确定、容错率高的任务。
    *   动态 Replanning 是应对不确定性（网络波动、逻辑悖论）的必选项。

2.  **规则 vs 语义**：
    *   在 Agent 开发初期，正则匹配（Rule-based）简单高效。
    *   但随着任务复杂度提升，必须引入语义判断（LLM-based），让 Agent 真正“理解”自己在做什么，而不是仅仅匹配字符串。

3.  **分层治理**：
    *   **Planner**：负责宏观战略。
    *   **Executor**：负责微观战术。
    *   **Critic**：负责质量管控。
    *   这种三权分立的架构是构建稳健智能体系统的基石。

---

今天的探索非常有价值，我们不仅复现了经典论文的范式，更通过实战调试发现并解决了真实落地中会遇到的核心难题。

您可以放心休息了，代码库现在处于一个非常整洁且功能强大的状态！ 🌙✨

## 计算器工具搭建与调试

### 搭建“计算器”工具

In [ ]:
import math

def calculator(expression: str) -> str:
    """
    一个数学计算器。能够执行基本的数学运算。
    输入参数应为数学表达式字符串，例如 "(123 + 456) * 789 / 12"。
    """
    try:
        # 1. 预处理：处理常见的非标准符号
        clean_expression = expression.replace("×", "*").replace("÷", "/")
        
        # 2. 安全性检查（简单版）：只允许数字和基本运算符
        allowed_chars = set("0123456789+-*/(). math.sqrt")
        if not all(c in allowed_chars or c.isspace() for c in clean_expression):
             return "Error: 表达式包含非法字符，仅支持基本数学运算。"

        # 3. 执行计算
        # 注意：生产环境建议使用更安全的库如 numexpr 或 llm-math，而非 eval
        result = eval(clean_expression, {"__builtins__": None}, {"math": math})
        return str(result)
        
    except Exception as e:
        return f"计算错误: {str(e)}"


In [ ]:
# 注册工具
# toolExecutor.registerTool("Calculator", "支持加减乘除及括号的数学计算器。输入参数 {'expression': '数学表达式'}", calculator)

### 测试“计算器”工具

In [ ]:
# 加载依赖包
import os
from openai import OpenAI
from dotenv import load_dotenv

# 加载 .env 文件 (确保能读取到根目录的 .env)
load_dotenv() 

# 加载工具函数
from llm_client import HelloAgentsLLM
from tools import ToolExecutor, calculator

In [ ]:
# 初始化工具执行器
toolExecutor = ToolExecutor()
toolExecutor.registerTool("Calculator", "...", calculator) 

# 初始化大语言模型客户端
llm_client=HelloAgentsLLM()

工具 'Calculator' 已注册。


In [16]:
import ReAct

agent = ReAct.ReActAgent(llm_client=llm_client, tool_executor=toolExecutor)

# 1. 测试复杂计算能力
agent.run("计算 (123 + 456) * 789 / 12 的结果")


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


'38069.25'

In [17]:
# 2. 测试错误引导 (故意使用错误的参数名，观察 Agent 是否自我修正)
# 例如，LLM 可能会错误地输出 {"expression": ...} 以外的键，系统应能提示正确签名。
agent.run("搜索 (123 + 456) × 789 / 12 的结果，结果保留两位小数")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


'38069.25'